**Задание:**

В процессе работы с аномальными начислениями потребителям тепловой энергии были выявлены случаи ошибочно внесённых показаний, а также случаи аномально низкого потребления тепловой энергии. Это привело к неправильным начислениям на общую сумму T млн рублей в пользу потребителя. Необходимо разработать модель, которая будет анализировать данные о потреблении тепловой энергии и выявлять аномальные начисления. Модель должна учитывать различные факторы, такие как показания приборов учёта, договорные нагрузки, погодные условия и другие параметры, которые могут повлиять на потребление тепловой энергии. Модель должна автоматически обнаруживать аномалии в данных о потреблении и предоставлять информацию о них ответственным сотрудникам для дальнейшего анализа и принятия решений.

**Виды аномалий** по показаниям приборов учёта тепловой энергии, которые необходимо выявлять (кроме объектов с видом энергопотребления ГВС (централ):
1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);
2. равные значения показаний в течение нескольких расчетных периодов;
3. снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);
4. аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
- год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
- этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
- площадь (±10%),
- наличие ГВС ИТП (горячей воды, учитываемой тем же прибором).


In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Разметка датасета аномалиями

1) Разметить target = 1 по условию "равные значения показаний в течение нескольких расчетных периодов (выше включали только отопительный период)"

2) Разметить target = 1 по условию "снижение/рост показаний в отдельные месяцы по сравнению с показаниями за предыдущие периоды по данному объекту (с учётом фактической температуры наружного воздуха и количества отопительных дней в месяце);"

3) Разметить target = 1 по условию "аномально низкое/высокое (отклонение более 25%) потребление объекта в конкретном месяце по сравнению с аналогичными объектами (только для типов объекта «Многоквартирный дом») по критериям:
- год постройки (по группам до 1958 г., 1959-1989 гг., 1990-2000 гг., 2001-2010 гг., 2011-2024 гг.),
- этажность (по группам 1-2 этажа, 3-4 этажа, 5-9 этажей,10-12 этажей, 13 и более этажей),
- площадь (±10%),
- наличие ГВС ИТП (горячей воды, учитываемой тем же прибором). "

In [3]:
pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Октябрь 2022 Уфа.xlsx").head(5)

,Дата формирования отчета: 06.09.2024 13:49:11; Сформировала: Игнатьева Ксения Фагимовна,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Подразделение,№ ОДПУ,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания,"Текущее потребление, Гкал"
1,Уфа,787,ГВС-ИТП,г Уфа,Другое строение,2022-11-01 00:00:00,141
2,Уфа,00119540,ГВС-ИТП,"г Уфа, б-р. Баландина, д.11",Многоквартирный дом,2022-11-01 00:00:00,81.25
3,Уфа,30822,ГВС-ИТП,"г Уфа, б-р. Баландина, д.2",Многоквартирный дом,2022-11-01 00:00:00,179.096
4,Уфа,237959,NaN,"г Уфа, б-р. Баландина, д.2а",Другое строение,2022-11-01 00:00:00,7.196


In [4]:
# Считываем таблицы из файлов excel отопительного периода, не пропуская строк, удаляем столбец "подразделение", переименовываем "№ ОДПУ" в "index"

# Октябрь
oct_21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Октябрь 2021 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
oct_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Октябрь 2022 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Ноябрь
nov_21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Ноябрь 2021 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
nov_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Ноябрь 2022 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Декабрь
dec_21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Декабрь 2021 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
dec_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Декабрь 2022 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Январь
jan_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Январь 2022 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
jan_23 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Январь 2023 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Феварль
feb_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Февраль 2022 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
feb_23 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Февраль 2023 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Март
mar_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Март 2022 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
mar_23 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Март 2023 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Апрель
apr_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Апрель 2022 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
apr_23 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Апрель 2023 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

In [5]:
#объединяем все таблицы отопительного периода в одну:
# столбцы "индекс", "адрес", "Вид энерг-а ГВС", "Тип объекта" объединяем
# и добавляем к новым столбцам соответствующие суффиксы

# DF в отопительный период (октябрь-апрель)
df = oct_21.merge(oct_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ("_21_10","_22_10"))
df = df.merge(nov_21, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_21_11"))
df = df.merge(nov_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_11"))
df = df.merge(dec_21, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_21_12"))
df = df.merge(dec_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_12"))
df = df.merge(jan_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_01"))
df = df.merge(jan_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_01"))
df = df.merge(feb_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_02"))
df = df.merge(feb_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_02"))
df = df.merge(mar_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_03"))
df = df.merge(mar_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_03"))
df = df.merge(apr_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_04"))
df = df.merge(apr_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_04"))

df['target'] = 0
df

,index,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания_21_10,"Текущее потребление, Гкал_21_10",Дата текущего показания_22_10,"Текущее потребление, Гкал_22_10",Дата текущего показания,"Текущее потребление, Гкал",...,"Текущее потребление, Гкал_23_02",Дата текущего показания_22_03,"Текущее потребление, Гкал_22_03",Дата текущего показания_23_03,"Текущее потребление, Гкал_23_03",Дата текущего показания_22_04,"Текущее потребление, Гкал_22_04",Дата текущего показания_23_04,"Текущее потребление, Гкал_23_04",target
0,00000473,NaN,"г Уфа, ул. Калинина, д.14",Многоквартирный дом,2021-11-01,36.949,2022-11-01,51.314,2021-12-01,49.4230,...,70.161,2022-04-01,64.462,2023-04-01,55.353,2022-05-01,49.4160,2023-05-01,46.538,0
1,"00004340149, 00004340150",NaN,"г Уфа, ул. Академика Королева, д.10 корп.4",Многоквартирный дом,2021-11-01,75.290,2022-11-01,106.610,2021-12-01,117.7300,...,144.980,2022-04-01,128.590,2023-04-01,114.730,2022-05-01,76.8800,2023-05-01,76.120,0
2,000101707,NaN,"г Уфа, ул. Кустарная, д.18, Подобъект №60352","Административные здания, конторы",2021-11-01,43.966,2022-11-01,34.134,2021-12-01,71.9460,...,61.814,2022-04-01,92.973,2023-04-01,62.110,2022-05-01,36.9010,2023-05-01,38.861,0
3,00012577,NaN,"г Уфа, ул. Рихарда Зорге, д.25 корп.2",Другое строение,2021-11-01,68.729,2022-11-01,71.891,2021-12-01,93.5150,...,123.489,2022-04-01,98.478,2023-04-01,81.835,2022-05-01,77.4620,2023-05-01,74.091,0
4,00012623,NaN,"г Уфа, ул. Харьковская, д.101",Многоквартирный дом,2021-11-01,38.294,2022-11-01,26.369,2021-12-01,54.2098,...,63.003,2022-04-01,60.012,2023-04-01,55.182,2022-05-01,46.5241,2023-05-01,31.190,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4652,CE260,NaN,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,46.200,2022-11-01,56.890,NaT,NaN,...,111.730,NaT,NaN,2023-04-01,85.210,2022-05-01,62.1600,2023-05-01,42.620,0
4653,CG074,ГВС-ИТП,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,32.701,2022-11-01,63.570,NaT,NaN,...,116.150,NaT,NaN,2023-04-01,90.700,2022-05-01,75.3400,2023-05-01,51.580,0
4654,"CG166, CF944",NaN,"г Уфа, ул. Трамвайная",Другое строение,2021-11-01,3449.290,2022-11-01,3458.510,2021-12-01,6094.2100,...,9493.260,2022-04-01,8062.370,2023-04-01,6723.740,2022-05-01,2502.7700,2023-05-01,2975.390,0
4655,MB152,NaN,"г Уфа, проезд. Лесной, д.3 корп.а",Другое строение,2021-11-01,312.920,2022-11-01,312.920,NaT,NaN,...,456.830,2022-04-01,837.510,2023-04-01,340.630,2022-05-01,470.5500,2023-05-01,208.500,0


In [6]:
df = df.rename(columns={'Текущее потребление, Гкал': 'Текущее потребление, Гкал_21_11'})

consumption_columns = [col for col in df.columns if col.startswith('Текущее потребление')]
consumption_columns.sort() #сортируем столбцы "по возрастанию"

consumption_columns

['Текущее потребление, Гкал_21_10',
 'Текущее потребление, Гкал_21_11',
 'Текущее потребление, Гкал_21_12',
 'Текущее потребление, Гкал_22_01',
 'Текущее потребление, Гкал_22_02',
 'Текущее потребление, Гкал_22_03',
 'Текущее потребление, Гкал_22_04',
 'Текущее потребление, Гкал_22_10',
 'Текущее потребление, Гкал_22_11',
 'Текущее потребление, Гкал_22_12',
 'Текущее потребление, Гкал_23_01',
 'Текущее потребление, Гкал_23_02',
 'Текущее потребление, Гкал_23_03',
 'Текущее потребление, Гкал_23_04']

In [7]:
# Тип аномалии = 1. нулевые значения показаний за тепловую энергию в отопительный период (октябрь-апрель);
df['target'] = np.where(((df[consumption_columns] == 0) | df[consumption_columns].isna()).any(axis=1), 1, df['target'])

In [ ]:
# Тип аномалии = 2. равные значения показаний за тепловую энергию в отопительный период (октябрь-апрель);
def check_same_values(row, min_months=2):
    equal_count = 1
    for i in range(len(consumption_columns) - 1):
        if row[consumption_columns[i]] == row[consumption_columns[i+1]]:
            equal_count += 1
            if equal_count >= min_months:
                return 1
        else:
            equal_count = 1
    return 0
df['target'] = df.apply(check_same_values, axis=1)

In [8]:
# Тип аномалии = 2. равные значения показаний за тепловую энергию в отопительный период (октябрь-апрель);
# 2 способ для быстрой работы
df['target'] = (df[consumption_columns].diff(axis=1) == 0).any(axis=1).astype(int)

In [9]:
rows_with_target_1 = df[df['target'] == 1] # выделяем из таблицы строки, где были найдены аномалии
rows_with_target_1

,index,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания_21_10,"Текущее потребление, Гкал_21_10",Дата текущего показания_22_10,"Текущее потребление, Гкал_22_10",Дата текущего показания,"Текущее потребление, Гкал_21_11",...,"Текущее потребление, Гкал_23_02",Дата текущего показания_22_03,"Текущее потребление, Гкал_22_03",Дата текущего показания_23_03,"Текущее потребление, Гкал_23_03",Дата текущего показания_22_04,"Текущее потребление, Гкал_22_04",Дата текущего показания_23_04,"Текущее потребление, Гкал_23_04",target
149,00108875,NaN,"г Уфа, ул. Блюхера, д.23 корп.2",Многоквартирный дом,2021-11-01,58.118,2022-11-01,70.933,2021-12-01,86.377,...,76.981000,2022-04-01,75.389,2023-04-01,60.393,2022-05-01,59.634,2023-05-01,52.708,1
292,"00255030, 254970",ГВС-ИТП,"г Уфа, пр-кт Октября, д.85",Многоквартирный дом,2021-11-01,154.018,2022-11-01,79.394,2021-12-01,169.377,...,173.171000,2022-04-01,157.918,2023-04-01,138.909,2022-05-01,131.821,2023-05-01,110.883,1
1516,1169,NaN,"г Уфа, ул. Новоженова, д.9 корп.1",Многоквартирный дом,2021-11-01,46.730,2022-11-01,46.730,2021-12-01,46.730,...,46.730000,2022-04-01,46.730,2023-04-01,46.730,2022-05-01,46.730,2023-05-01,37.384,1
1688,13-009451,ГВС-ИТП,"г Уфа, ул. Российская, д.20",Многоквартирный дом,2021-11-01,213.104,2022-11-01,416.879,2021-12-01,302.388,...,406.048000,2022-04-01,430.775,2023-04-01,386.209,2022-05-01,225.817,2023-05-01,210.356,1
1779,13493,ГВС-ИТП,"г Уфа, ул. 50-летия Октября, д.24",Другое строение,NaT,NaN,NaT,NaN,2021-12-01,0.000,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1
1984,15-022815,NaN,"г Уфа, пр-кт Октября, д.124 корп.2",Многоквартирный дом,2021-11-01,26.730,2022-11-01,17.760,2021-12-01,39.860,...,47.930760,2022-04-01,37.319,2023-04-01,36.384,2022-05-01,25.746,2023-05-01,23.099,1
2080,1524327,NaN,"г Уфа, ул. Энтузиастов, д.14, Подобъект №984513",Многоквартирный дом,NaT,NaN,2022-11-01,0.800,NaT,NaN,...,3.000000,2022-04-01,1.000,NaT,NaN,2022-05-01,1.700,NaT,NaN,1
2529,17470191,NaN,"г Уфа, ул. Комсомольская, д.15, Подобъект №35951",Многоквартирный дом,2021-11-01,0.000,NaT,NaN,2021-12-01,0.500,...,NaN,2022-04-01,2.000,2023-04-01,2.000,2022-05-01,1.000,2023-05-01,1.400,1
2538,176464ГВС,ГВС (централ),"г Уфа, ул. Краснодонская, д.3",Многоквартирный дом,2021-11-01,58.079,NaT,NaN,2021-12-01,58.786,...,0.000000,2022-04-01,64.942,2023-04-01,0.000,2022-05-01,70.804,2023-05-01,0.000,1
2586,18-028134,NaN,"г Уфа, ул. Маршала Жукова, д.1/2, Подобъект №9...",Многоквартирный дом,NaT,NaN,NaT,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1


In [10]:
# импортируем таблицы из файлов excel неотопительного периода, не пропуская строк, удаляем столбец "подразделение", переименовываем "№ ОДПУ" в "index"

# Август
aug_21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Август 2021 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
aug_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Август 2022 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Сентябрь
sen_21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Сентябрь 2021 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
sen_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Сентябрь 2022 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Май
may_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Май 2022 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
may_23 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Май 2023 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Июнь
jun_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Июнь 2022 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
jun_23 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Июнь 2023 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

# Июль
jul_21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Июль 2021 Уфа.xls", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})
jul_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Июль 2022 Уфа.xlsx", skiprows= lambda x: x == 0).drop(['Подразделение'], axis =1).rename(columns={'№ ОДПУ': 'index'})

In [11]:
# DF2 за весь период
df2 = oct_21.merge(oct_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ("_21_10","_22_10"))
df2 = df2.merge(nov_21, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_21_11"))
df2 = df2.merge(nov_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_11"))
df2 = df2.merge(dec_21, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_21_12"))
df2 = df2.merge(dec_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_12"))
df2 = df2.merge(jan_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_01"))
df2 = df2.merge(jan_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_01"))
df2 = df2.merge(feb_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_02"))
df2 = df2.merge(feb_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_02"))
df2 = df2.merge(mar_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_03"))
df2 = df2.merge(mar_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_03"))
df2 = df2.merge(apr_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_04"))
df2 = df2.merge(apr_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_04"))
df2 = df2.merge(aug_21, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_21_08"))
df2 = df2.merge(aug_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_08"))
df2 = df2.merge(sen_21, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_21_09"))
df2 = df2.merge(sen_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_09"))
df2 = df2.merge(may_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_05"))
df2 = df2.merge(may_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_05"))
df2 = df2.merge(jun_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_06"))
df2 = df2.merge(jun_23, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_23_06"))
df2 = df2.merge(jul_21, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_21_07"))
df2 = df2.merge(jul_22, on=["index", "Адрес объекта", "Вид энерг-а ГВС", "Тип объекта"], how="outer", suffixes = ('', "_22_07"))

df2['target'] = 0
df2

,index,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания_21_10,"Текущее потребление, Гкал_21_10",Дата текущего показания_22_10,"Текущее потребление, Гкал_22_10",Дата текущего показания,"Текущее потребление, Гкал",...,"Текущее потребление, Гкал_23_05",Дата текущего показания_22_06,"Текущее потребление, Гкал_22_06",Дата текущего показания_23_06,"Текущее потребление, Гкал_23_06",Дата текущего показания_21_07,"Текущее потребление, Гкал_21_07",Дата текущего показания_22_07,"Текущее потребление, Гкал_22_07",target
0,00000473,NaN,"г Уфа, ул. Калинина, д.14",Многоквартирный дом,2021-11-01,36.949,2022-11-01,51.314,2021-12-01,49.4230,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
1,"00004340149, 00004340150",NaN,"г Уфа, ул. Академика Королева, д.10 корп.4",Многоквартирный дом,2021-11-01,75.290,2022-11-01,106.610,2021-12-01,117.7300,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
2,000101707,NaN,"г Уфа, ул. Кустарная, д.18, Подобъект №60352","Административные здания, конторы",2021-11-01,43.966,2022-11-01,34.134,2021-12-01,71.9460,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
3,00012577,NaN,"г Уфа, ул. Рихарда Зорге, д.25 корп.2",Другое строение,2021-11-01,68.729,2022-11-01,71.891,2021-12-01,93.5150,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
4,00012623,NaN,"г Уфа, ул. Харьковская, д.101",Многоквартирный дом,2021-11-01,38.294,2022-11-01,26.369,2021-12-01,54.2098,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666,CE260,NaN,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,46.200,2022-11-01,56.890,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
4667,CG074,ГВС-ИТП,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,32.701,2022-11-01,63.570,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
4668,"CG166, CF944",NaN,"г Уфа, ул. Трамвайная",Другое строение,2021-11-01,3449.290,2022-11-01,3458.510,2021-12-01,6094.2100,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0
4669,MB152,NaN,"г Уфа, проезд. Лесной, д.3 корп.а",Другое строение,2021-11-01,312.920,2022-11-01,312.920,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,0


In [12]:
df2 = df2.rename(columns={'Текущее потребление, Гкал': 'Текущее потребление, Гкал_21_11'})

consumption_columns2 = [col for col in df2.columns if col.startswith('Текущее потребление')]
consumption_columns2.sort()

consumption_columns2

['Текущее потребление, Гкал_21_07',
 'Текущее потребление, Гкал_21_08',
 'Текущее потребление, Гкал_21_09',
 'Текущее потребление, Гкал_21_10',
 'Текущее потребление, Гкал_21_11',
 'Текущее потребление, Гкал_21_12',
 'Текущее потребление, Гкал_22_01',
 'Текущее потребление, Гкал_22_02',
 'Текущее потребление, Гкал_22_03',
 'Текущее потребление, Гкал_22_04',
 'Текущее потребление, Гкал_22_05',
 'Текущее потребление, Гкал_22_06',
 'Текущее потребление, Гкал_22_07',
 'Текущее потребление, Гкал_22_08',
 'Текущее потребление, Гкал_22_09',
 'Текущее потребление, Гкал_22_10',
 'Текущее потребление, Гкал_22_11',
 'Текущее потребление, Гкал_22_12',
 'Текущее потребление, Гкал_23_01',
 'Текущее потребление, Гкал_23_02',
 'Текущее потребление, Гкал_23_03',
 'Текущее потребление, Гкал_23_04',
 'Текущее потребление, Гкал_23_05',
 'Текущее потребление, Гкал_23_06']

In [13]:
# нулевые значения показаний за тепловую энергию за отопительный сезон для таблицы DF2;
df2['target'] = np.where(((df2[consumption_columns] == 0) | df2[consumption_columns].isna()).any(axis=1), 1, df2['target'])

In [14]:
df2[consumption_columns] = df2[consumption_columns].fillna(0)

In [ ]:
# равные значения показаний за тепловую энергию за весь период;
def check_same_values(row):
    for i in range(len(consumption_columns2) - 1):
          if row[consumption_columns2[i]] == row[consumption_columns2[i+1]]:
              return 1
    return 0

df2['target'] = df2.apply(check_same_values, axis=1)

In [15]:
# 2 способ для быстрой работы
df2['target'] = (df2[consumption_columns2].diff(axis=1) == 0).any(axis=1).astype(int)

In [16]:
rows_with_target_1__2 = df2[df2['target'] == 1]
rows_with_target_1__2

,index,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания_21_10,"Текущее потребление, Гкал_21_10",Дата текущего показания_22_10,"Текущее потребление, Гкал_22_10",Дата текущего показания,"Текущее потребление, Гкал_21_11",...,"Текущее потребление, Гкал_23_05",Дата текущего показания_22_06,"Текущее потребление, Гкал_22_06",Дата текущего показания_23_06,"Текущее потребление, Гкал_23_06",Дата текущего показания_21_07,"Текущее потребление, Гкал_21_07",Дата текущего показания_22_07,"Текущее потребление, Гкал_22_07",target
5,00014847,ГВС-ИТП,"г Уфа, ул. Достоевского, д.106",Многоквартирный дом,2021-11-01,190.464,NaT,0.000,2021-12-01,214.775,...,NaN,NaT,NaN,NaT,NaN,2021-08-01,32.64,NaT,NaN,1
8,00048959,NaN,"г Уфа, ул. Ленина, д.70",Другое строение,2021-11-01,2.379,NaT,0.000,2021-12-01,28.210,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1
9,00050593,NaN,"г Уфа, ул. Заводская, д.13",Другое строение,2021-11-01,8.727,NaT,0.000,2021-12-01,12.563,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1
21,00065243,NaN,"г Уфа, б-р. Хадии Давлетшиной, д.21, Подобъект...",Гаражи,NaT,0.000,NaT,0.000,NaT,0.000,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1
31,00072506,NaN,"г Уфа, ул. Малая Шелководная, д.3","Учебное заведение, комбинат, центр",2021-11-01,49.741,2022-11-01,49.404,2021-12-01,89.716,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4661,9926,NaN,"г Уфа, проезд. Лесной, д.8 корп.3",Другое строение,2021-11-01,15.585,NaT,0.000,2021-12-01,24.877,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1
4665,999305,ГВС-ИТП,"г Уфа, ул. Элеваторная, д.17",Другое строение,2021-11-01,324.000,2022-11-01,341.000,2021-12-01,362.000,...,70.0,2022-07-01,64.0,2023-07-01,64.0,2021-08-01,55.00,2022-08-01,60.0,1
4666,CE260,NaN,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,46.200,2022-11-01,56.890,NaT,0.000,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1
4667,CG074,ГВС-ИТП,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,32.701,2022-11-01,63.570,NaT,0.000,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1


In [17]:
temp2 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Температуры, продолжительность ОП.xls")
temp2.head()

,Unnamed: 0,Климатические параметры отопительного периода по г.Уфа,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,Период,2021-07-01,2021-08-01,2021-09-01,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00,2022-01-01 00:00:00,2022-02-01 00:00:00,...,2022-09-01,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01,2023-06-01
1,NaN,"Тн.в, град.С",NaT,NaT,NaT,6.696552,-1.0,-5.176667,-11.5,-6.735714,...,NaT,7.137037,-1.606452,-10.903333,-12.745161,-8.944828,-2.266667,7.81875,NaT,NaT
2,NaN,"Продолжительность ОЗП, сут.",NaT,NaT,NaT,29,31,30,31,28,...,NaT,27,31,30,31,29,30,32,NaT,NaT


In [18]:
#готовим данные по температуре и продолжительности

columns_to_drop = ['Unnamed: 0', 'Климатические параметры отопительного периода по г.Уфа',  'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
                   'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
                   'Unnamed: 16', 'Unnamed: 24', 'Unnamed: 25']
temp2.drop(columns=columns_to_drop, inplace=True)

temp2.columns = temp2.iloc[0] # берем первую строку в качестве названий столбцов
temp2 = temp2[1:] # удаляем эту строку из данных

temp2.reset_index(drop=True, inplace=True) #сбрасываем индексы строк таблицы, чтобы они шли по возрастанию

temp2.head()

/usr/local/lib/python3.11/dist-packages/pandas/core/indexes/base.py:7631: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(index_like, name=name, copy=copy)


,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-10-01,2022-11-01,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01
0,6.696552,-1.0,-5.176667,-11.5,-6.735714,-5.151613,5.65,7.137037,-1.606452,-10.903333,-12.745161,-8.944828,-2.266667,7.81875
1,29,31,30,31,28,31,34,27,31,30,31,29,30,32


In [19]:
def detect_temp_anomalies(df, consumption_cols, temp_data):

    for i in range(1, len(consumption_cols)):
        current_col = consumption_cols[i]
        prev_col = consumption_cols[i-1]

        # получаем метеоданные
        temp_current = temp_data.iloc[0, i]
        temp_prev = temp_data.iloc[0, i-1]
        days_current = temp_data.iloc[1, i]
        days_prev = temp_data.iloc[1, i-1]

        # рассчитываем изменения
        temp_change = temp_current - temp_prev

        # нормированное потребление (Гкал/день)
        df['norm_current'] = df[current_col] / days_current if days_current != 0 else 0
        df['norm_prev'] = df[prev_col] / days_prev if days_prev != 0 else 0
        consumption_change = df['norm_current'] - df['norm_prev']

        # условие аномалии
        anomaly_mask = (
            (consumption_change > 0) & (temp_change > 0) |
            (consumption_change < 0) & (temp_change < 0) )

        # помечаем аномалии
        df.loc[anomaly_mask, 'target'] = 1

    return df


df2 = detect_temp_anomalies(df2, consumption_columns, temp2)

In [20]:
anomaly_count = df2[df2['target'] == 1].shape[0] #считаем количество найденных аномалий
anomaly_count #выводим количество аномалий

3321

In [21]:
rows_with_target_1__2 = df2[df2['target'] == 1] # выделяем из таблицы строки, где были найдены аномалии
rows_with_target_1__2 #выводим аномалии

,index,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания_21_10,"Текущее потребление, Гкал_21_10",Дата текущего показания_22_10,"Текущее потребление, Гкал_22_10",Дата текущего показания,"Текущее потребление, Гкал_21_11",...,"Текущее потребление, Гкал_22_06",Дата текущего показания_23_06,"Текущее потребление, Гкал_23_06",Дата текущего показания_21_07,"Текущее потребление, Гкал_21_07",Дата текущего показания_22_07,"Текущее потребление, Гкал_22_07",target,norm_current,norm_prev
0,00000473,NaN,"г Уфа, ул. Калинина, д.14",Многоквартирный дом,2021-11-01,36.949,2022-11-01,51.314,2021-12-01,49.4230,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,1.454312,1.845100
1,"00004340149, 00004340150",NaN,"г Уфа, ул. Академика Королева, д.10 корп.4",Многоквартирный дом,2021-11-01,75.290,2022-11-01,106.610,2021-12-01,117.7300,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,2.378750,3.824333
2,000101707,NaN,"г Уфа, ул. Кустарная, д.18, Подобъект №60352","Административные здания, конторы",2021-11-01,43.966,2022-11-01,34.134,2021-12-01,71.9460,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,1.214406,2.070333
3,00012577,NaN,"г Уфа, ул. Рихарда Зорге, д.25 корп.2",Другое строение,2021-11-01,68.729,2022-11-01,71.891,2021-12-01,93.5150,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,2.315344,2.727833
4,00012623,NaN,"г Уфа, ул. Харьковская, д.101",Многоквартирный дом,2021-11-01,38.294,2022-11-01,26.369,2021-12-01,54.2098,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,0.974688,1.839400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4665,999305,ГВС-ИТП,"г Уфа, ул. Элеваторная, д.17",Другое строение,2021-11-01,324.000,2022-11-01,341.000,2021-12-01,362.0000,...,64.0,2023-07-01,64.0,2021-08-01,55.0,2022-08-01,60.0,1,8.406250,13.100000
4666,CE260,NaN,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,46.200,2022-11-01,56.890,NaT,0.0000,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,1.331875,2.840333
4667,CG074,ГВС-ИТП,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,32.701,2022-11-01,63.570,NaT,0.0000,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,1.611875,3.023333
4668,"CG166, CF944",NaN,"г Уфа, ул. Трамвайная",Другое строение,2021-11-01,3449.290,2022-11-01,3458.510,2021-12-01,6094.2100,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,1,92.980937,224.124667


In [22]:
houses_df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/task#3/Тип строения, этажность, площадь, год постройки.xlsx")
houses_df.head(5)

,Дата формирования отчета: 06.09.2024 12:38:46; Сформировала: Игнатьева Ксения Фагимовна,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
1,"г Уфа, б-р. Баландина, д.11",Многоквартирный дом,18,2011-01-01 00:00:00,5394.5
2,"г Уфа, б-р. Баландина, д.2",Многоквартирный дом,16,2012-01-01 00:00:00,12731.5
3,"г Уфа, б-р. Баландина, д.2а",Другое строение,3,NaN,0
4,"г Уфа, б-р. Баландина, д.4",Многоквартирный дом,16,2012-01-01 00:00:00,4234.9


In [23]:
houses_df.columns = houses_df.iloc[0] # берем первую строку в качестве названий столбцов
houses_df = houses_df[1:] # удаляем эту строку из данных
houses_df.reset_index(drop=True, inplace=True) #сбрасываем индексы строк таблицы, чтобы они шли по возрастанию
houses_df.head(5)

,Адрес объекта,Тип Объекта,Этажность объекта,Дата постройки,Общая площадь объекта
0,"г Уфа, б-р. Баландина, д.11",Многоквартирный дом,18,2011-01-01 00:00:00,5394.5
1,"г Уфа, б-р. Баландина, д.2",Многоквартирный дом,16,2012-01-01 00:00:00,12731.5
2,"г Уфа, б-р. Баландина, д.2а",Другое строение,3,NaN,0
3,"г Уфа, б-р. Баландина, д.4",Многоквартирный дом,16,2012-01-01 00:00:00,4234.9
4,"г Уфа, б-р. Баландина, д.4 корп.1",Многоквартирный дом,16,2012-01-01 00:00:00,4250.1


In [24]:
# предварительная обработка
houses_df['Дата постройки'] = pd.to_datetime(houses_df['Дата постройки'], errors='coerce')
houses_df['Этажность объекта'] = pd.to_numeric(houses_df['Этажность объекта'], errors='coerce')
houses_df['Общая площадь объекта'] = pd.to_numeric(houses_df['Общая площадь объекта'], errors='coerce')


In [27]:
df2 #выводим объединенную таблицу

,index,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания_21_10,"Текущее потребление, Гкал_21_10",Дата текущего показания_22_10,"Текущее потребление, Гкал_22_10",Дата текущего показания,"Текущее потребление, Гкал_21_11",...,"Текущее потребление, Гкал_23_02",Дата текущего показания_22_03,"Текущее потребление, Гкал_22_03",Дата текущего показания_23_03,"Текущее потребление, Гкал_23_03",Дата текущего показания_22_04,"Текущее потребление, Гкал_22_04",Дата текущего показания_23_04,"Текущее потребление, Гкал_23_04",target
0,00000473,NaN,"г Уфа, ул. Калинина, д.14",Многоквартирный дом,2021-11-01,36.949,2022-11-01,51.314,2021-12-01,49.4230,...,70.161,2022-04-01,64.462,2023-04-01,55.353,2022-05-01,49.4160,2023-05-01,46.538,0
1,"00004340149, 00004340150",NaN,"г Уфа, ул. Академика Королева, д.10 корп.4",Многоквартирный дом,2021-11-01,75.290,2022-11-01,106.610,2021-12-01,117.7300,...,144.980,2022-04-01,128.590,2023-04-01,114.730,2022-05-01,76.8800,2023-05-01,76.120,0
2,000101707,NaN,"г Уфа, ул. Кустарная, д.18, Подобъект №60352","Административные здания, конторы",2021-11-01,43.966,2022-11-01,34.134,2021-12-01,71.9460,...,61.814,2022-04-01,92.973,2023-04-01,62.110,2022-05-01,36.9010,2023-05-01,38.861,0
3,00012577,NaN,"г Уфа, ул. Рихарда Зорге, д.25 корп.2",Другое строение,2021-11-01,68.729,2022-11-01,71.891,2021-12-01,93.5150,...,123.489,2022-04-01,98.478,2023-04-01,81.835,2022-05-01,77.4620,2023-05-01,74.091,0
4,00012623,NaN,"г Уфа, ул. Харьковская, д.101",Многоквартирный дом,2021-11-01,38.294,2022-11-01,26.369,2021-12-01,54.2098,...,63.003,2022-04-01,60.012,2023-04-01,55.182,2022-05-01,46.5241,2023-05-01,31.190,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4652,CE260,NaN,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,46.200,2022-11-01,56.890,NaT,NaN,...,111.730,NaT,NaN,2023-04-01,85.210,2022-05-01,62.1600,2023-05-01,42.620,0
4653,CG074,ГВС-ИТП,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,32.701,2022-11-01,63.570,NaT,NaN,...,116.150,NaT,NaN,2023-04-01,90.700,2022-05-01,75.3400,2023-05-01,51.580,0
4654,"CG166, CF944",NaN,"г Уфа, ул. Трамвайная",Другое строение,2021-11-01,3449.290,2022-11-01,3458.510,2021-12-01,6094.2100,...,9493.260,2022-04-01,8062.370,2023-04-01,6723.740,2022-05-01,2502.7700,2023-05-01,2975.390,0
4655,MB152,NaN,"г Уфа, проезд. Лесной, д.3 корп.а",Другое строение,2021-11-01,312.920,2022-11-01,312.920,NaT,NaN,...,456.830,2022-04-01,837.510,2023-04-01,340.630,2022-05-01,470.5500,2023-05-01,208.500,0


In [29]:
def assign_group(row): # создаем функцию для формирования групп зданий
# формируем группы по году постройки
    if pd.isna(row['Дата постройки']):
        year_group = 6  # в случае, если год не указан в таблице
    elif row['Дата постройки'].year <= 1958:
        year_group = 1
    elif 1959 <= row['Дата постройки'].year <= 1989:
        year_group = 2
    elif 1990 <= row['Дата постройки'].year <= 2000:
        year_group = 3
    elif 2001 <= row['Дата постройки'].year <= 2010:
        year_group = 4
    else:
        year_group = 5
# формируем группы по этажности объекта
    if row['Этажность объекта'] in [1, 2]:
        floor_group = 1
    elif row['Этажность объекта'] in [3, 4]:
        floor_group = 2
    elif 5 <= row['Этажность объекта'] <= 9:
        floor_group = 3
    elif 10 <= row['Этажность объекта'] <= 12:
        floor_group = 4
    elif row['Этажность объекта'] >= 13:
        floor_group = 5
    else:
        floor_group = 0  # в случае, если этажность не указана. присваиваем 0
# формируем группу по площади объекта
    area_group = 1 if row['Общая площадь объекта'] > 0 else 2

# формируем уникауникальный индекс для каждой группы
    group_index = f"{year_group}{floor_group}{area_group}"
    return int(group_index)

houses_df['group'] = houses_df.apply(assign_group, axis=1)

# объединяем таблицу по всем периодам
df2 = df2.merge(houses_df[['Адрес объекта', 'Этажность объекта', 'Общая площадь объекта', 'Дата постройки', 'group']], on='Адрес объекта', how='inner', suffixes=('', '_house'))

def adjust_group(row): # изменяем последнюю цифру группы в зависимости от наличия "ГВС-ИТП"
    base_group = int(row['group']) if pd.notna(row['group']) else 0 # приводим значение group к целому числу
    if row['Вид энерг-а ГВС'] == "ГВС-ИТП":
        return int(str(base_group) + '1') # Добавляем 1 в конец, если ГВС-ИТП есть
    return int(str(base_group) + '2') # Добавляем 2 в конец, если ГВС-ИТП нет

df2['group'] = df2.apply(adjust_group, axis=1) # применяем функцию для корректировки группы
print(df2[['Адрес объекта', 'group']].head())

                                Адрес объекта  group
0                   г Уфа, ул. Калинина, д.14   1212
1  г Уфа, ул. Академика Королева, д.10 корп.4   3412
2       г Уфа, ул. Рихарда Зорге, д.25 корп.2   6212
3               г Уфа, ул. Харьковская, д.101   2212
4              г Уфа, ул. Достоевского, д.106   2311


In [30]:
def detect_anomalies(df2):
    for index, row in df2.iterrows():
        if row['Тип объекта'] != 'Многоквартирный дом':
            continue
        # получаем группу, площадь и этажность текущего объекта
        current_group = row['group']
        current_area = row['Общая площадь объекта']
        current_floors = row['Этажность объекта']

        # фильтруем таблицу по группе и площади (±10%)
        similar_rows = df2[(df2['group'] == current_group) &
                          (df2['Общая площадь объекта'].between(current_area * 0.9, current_area * 1.1))
                          ]

        for _, similar_row in similar_rows.iterrows():
            if row.name == similar_row.name:
                continue

            for col in similar_row.index: # сравниваем потребление в данной группе по площади
                if col.startswith("Текущее потребление"):
                    current_consumption = row[col]
                    similar_consumption = similar_row[col]

                    if pd.notna(current_consumption) and pd.notna(similar_consumption):
                        deviation = abs(current_consumption - similar_consumption)

                        # проверяем на аномалии (отклонение больше 25%)
                        if deviation > 0.25 * current_consumption:
                            df2.at[index, 'target'] = 1
                            break

detect_anomalies(df2)

anomaly_count = df2[df2['target'] == 1].shape[0]
print(f"Количество аномалий: {anomaly_count}")

Количество аномалий: 3128


In [31]:
df2.loc[df2['Вид энерг-а ГВС'] == 'ГВС (централ)', 'target'] = 0 #обнуляем аномалии на адресах, где есть ГВС-централ

In [32]:
anomaly_count = df2[df2['target'] == 1].shape[0]
print(f"Количество аномалий: {anomaly_count}")

Количество аномалий: 3119


In [33]:
df2

,index,Вид энерг-а ГВС,Адрес объекта,Тип объекта,Дата текущего показания_21_10,"Текущее потребление, Гкал_21_10",Дата текущего показания_22_10,"Текущее потребление, Гкал_22_10",Дата текущего показания,"Текущее потребление, Гкал_21_11",...,"Текущее потребление, Гкал_23_03",Дата текущего показания_22_04,"Текущее потребление, Гкал_22_04",Дата текущего показания_23_04,"Текущее потребление, Гкал_23_04",target,Этажность объекта,Общая площадь объекта,Дата постройки,group
0,00000473,NaN,"г Уфа, ул. Калинина, д.14",Многоквартирный дом,2021-11-01,36.949,2022-11-01,51.314,2021-12-01,49.4230,...,55.353,2022-05-01,49.4160,2023-05-01,46.538,1,4.0,2319.20,1952-01-01,1212
1,"00004340149, 00004340150",NaN,"г Уфа, ул. Академика Королева, д.10 корп.4",Многоквартирный дом,2021-11-01,75.290,2022-11-01,106.610,2021-12-01,117.7300,...,114.730,2022-05-01,76.8800,2023-05-01,76.120,1,12.0,6795.80,2000-01-01,3412
2,00012577,NaN,"г Уфа, ул. Рихарда Зорге, д.25 корп.2",Другое строение,2021-11-01,68.729,2022-11-01,71.891,2021-12-01,93.5150,...,81.835,2022-05-01,77.4620,2023-05-01,74.091,0,3.0,0.01,NaT,6212
3,00012623,NaN,"г Уфа, ул. Харьковская, д.101",Многоквартирный дом,2021-11-01,38.294,2022-11-01,26.369,2021-12-01,54.2098,...,55.182,2022-05-01,46.5241,2023-05-01,31.190,1,3.0,2160.01,1960-01-01,2212
4,00014847,ГВС-ИТП,"г Уфа, ул. Достоевского, д.106",Многоквартирный дом,2021-11-01,190.464,NaT,NaN,2021-12-01,214.7750,...,NaN,NaT,NaN,NaT,NaN,1,5.0,6509.80,1968-01-01,2311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4765,CG074,ГВС-ИТП,"г Уфа, ул. Ульяновых, д.74",Другое строение,2021-11-01,32.701,2022-11-01,63.570,NaT,NaN,...,90.700,2022-05-01,75.3400,2023-05-01,51.580,0,2.0,0.01,NaT,6111
4766,"CG166, CF944",NaN,"г Уфа, ул. Трамвайная",Другое строение,2021-11-01,3449.290,2022-11-01,3458.510,2021-12-01,6094.2100,...,6723.740,2022-05-01,2502.7700,2023-05-01,2975.390,0,4.0,0.01,NaT,6212
4767,MB152,NaN,"г Уфа, проезд. Лесной, д.3 корп.а",Другое строение,2021-11-01,312.920,2022-11-01,312.920,NaT,NaN,...,340.630,2022-05-01,470.5500,2023-05-01,208.500,0,2.0,0.01,NaT,6112
4768,АН381,ГВС-ИТП,"г Уфа, ул. Машиностроителей, д.10",Другое строение,2021-11-01,112.640,2022-11-01,93.124,2021-12-01,142.6400,...,144.370,2022-05-01,122.0700,2023-05-01,110.480,0,5.0,0.01,1970-01-01,2311


In [52]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score

date_columns = df2.select_dtypes(include=['datetime64']).columns.tolist()
#print("DateTime Columns:", date_columns)
for col in date_columns:
  df2[col] = df2[col].astype(np.int64) // 10**9 # преобразуем дату и время в секунды

categorical_columns = df2.select_dtypes(include=['object']).columns.tolist() #выделяем столбцы с категориальными признаками (т.е не имеющими численного представления)
df_en = pd.get_dummies(df2, columns=categorical_columns, drop_first=True) #преобразовываем категориальные признаки в некую чиленную замену

X = df_en.drop(columns=['target'])  # выделили фичи
y = df_en['target']                 # выделили таргет

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) #разделяем набор данных на обучающую (0,8) и тестовую (0,2) выборки

model = DecisionTreeClassifier(max_depth = 7, min_samples_split = 5, min_samples_leaf = 5, random_state=42) #создаем древовидную модель
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       330
           1       0.97      0.99      0.98       624

    accuracy                           0.97       954
   macro avg       0.98      0.96      0.97       954
weighted avg       0.97      0.97      0.97       954



In [54]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    model, X, y, cv=5, scoring="f1"
)
print(f"F1-score (CV): {scores.mean():.3f} ± {scores.std():.3f}")

F1-score (CV): 0.976 ± 0.006


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

date_columns = df2. select_dtypes(include=[ 'datetime64']) .columns. tolist()
for col in date_columns:
  df2[col] = df2[col].astype(np.int64) // 10**9 # преобразуем дату и время в секунды

categorical_columns = df2.select_dtypes(include=['object']).columns.tolist() #выделяем столбцы с категориальными признаками (т.е не имеющими численного представления)
df_en = pd.get_dummies(df2, columns=categorical_columns, drop_first=True) #преобразовываем категориальные признаки в некую чиленную замену

X = df_en.drop(columns=['target'])  # фичи
y = df_en['target']                 # таргет

imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X) #заменяем отсутствующие значения средними

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingClassifier(learning_rate=0.05, n_estimators=800, max_depth=4)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.94      0.97       326
           1       0.97      1.00      0.98       628

    accuracy                           0.98       954
   macro avg       0.98      0.97      0.97       954
weighted avg       0.98      0.98      0.98       954



In [57]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score

date_columns = df2.select_dtypes(include=['datetime64']).columns.tolist()
#print("DateTime Columns:", date_columns)
for col in date_columns:
  df2[col] = df2[col].astype(np.int64) // 10**9 # преобразуем дату и время в секунды

categorical_columns = df2.select_dtypes(include=['object']).columns.tolist() #выделяем столбцы с категориальными признаками (т.е не имеющими численного представления)
df_en = pd.get_dummies(df2, columns=categorical_columns, drop_first=True) #преобразовываем категориальные признаки в некую чиленную замену

X = df_en.drop(columns=['target'])  # выделили фичи
y = df_en['target']                 # выделили таргет

imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X) #заменяем отсутствующие значения средними

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) #разделяем набор данных на обучающую (0,8) и тестовую (0,2) выборки


tree = DecisionTreeClassifier(max_depth=7, min_samples_leaf=5)
gb = GradientBoostingClassifier(learning_rate=0.05, n_estimators=800, max_depth=4)

ensemble = VotingClassifier(
    estimators=[("tree", tree), ("gb", gb)],
    voting="soft"
)
ensemble.fit(X_train, y_train)
y_pred_ensemble = ensemble.predict(X_test)
print(classification_report(y_test, y_pred_ensemble))

              precision    recall  f1-score   support

           0       0.99      0.92      0.96       330
           1       0.96      1.00      0.98       624

    accuracy                           0.97       954
   macro avg       0.98      0.96      0.97       954
weighted avg       0.97      0.97      0.97       954



In [58]:
print("Train F1:", f1_score(y_train, ensemble.predict(X_train)))

Train F1: 0.9906542056074766


In [60]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier


tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=10, random_state=42)
gb = HistGradientBoostingClassifier(max_iter=500, max_depth=3, l2_regularization=1.0, random_state=42)

stack = StackingClassifier(
    estimators=[("tree", tree), ("gb", gb)],
    final_estimator=LogisticRegression(C=0.1, max_iter=1000),
    cv=5
)
stack.fit(X_train, y_train)
print("Test F1:", f1_score(y_test, stack.predict(X_test)))

Test F1: 0.978756884343037


In [61]:
import joblib
joblib.dump(ensemble, 'best_ensemble_model.pkl')

['best_ensemble_model.pkl']